In [2]:
from mxpkg.lib.dsw_utils import notebook_init, log_level

# if running local (directory is relative to the directory of the notebook)
notebook_init(clay_config='../../../mxpkg/config/development.yaml')

# if using workbench
# notebook_init(clay_config='/home/udocker/.mxpkg/config/workbench.yaml')

from mxpkg.api.switchback import SwitchbackAnalysis
from mxpkg.services.gateways.mxbackend import MXBackendGateway
#log_level('error')


# from mxpkg.entities.analysis.segment import SegmentType, Segment
# from mxpkg.entities.analysis.switchback import (
#     SwitchbackExperimentAnalysisConfig, SwitchbackMetricAnalysisConfig,
#     SwitchbackClusteringModelConfig, SwitchbackExperimentConfig, ExperimentPhase)
# from mxpkg.services.analysis.switchback import SwitchbackAnalysisService
# from mxpkg.services.analysis.report_new import SwitchbackAnalysisReportService
# from mxpkg.services.metric_definitions import MetricDefinitionsService
# from mxpkg.stats.switchback_utils import CategoricalControls

from mxpkg.entities.analysis.switchback import *
from mxpkg.services.metric_definitions import MetricDefinitionsService


# replacing the above block to get data from outage time
from mxpkg.lib import constants
constants.ENABLE_LOCKDOWN = False
constants.ENABLE_LOCKDOWN_DRS = False
import pandas as pd
from mxpkg.api.synthetic import SyntheticControl
from mxpkg.entities.analysis.synthetic import (SyntheticVariable, SyntheticMetric,
    SyntheticExperimentConfigNew, SyntheticDataConfigNew, SyntheticAnalysisConfigNew)
from mxpkg.services.analysis.report_new import SyntheticAnalysisReportServiceNew


mxpkg version: 1.1.42
environment: dsw


In [13]:
def get_sb_for_segment_types(segment_types, segment_filters, metric_names):
    sb = SwitchbackAnalysis(
        metric_names = metric_names,
        experiment_name=xp_name,
        segment_types=segment_types,
        segment_filters=segment_filters,
        sparse_obs_filtering=False,
#         exclude_time_periods=[
#         [ '2021-08-25T09:00:00Z', '2021-08-25T19:00:00Z'],    # 'Z' indicates UTC
#     ],
    )
    sb.collect_data()
    return sb
def get_barchart(sb, segment_filters, metric_names):
    sb.results_barchart(
        metric_names=metric_names, 
        segment_filters=segment_filters,
        level=False, 
        figure_size=(15, 1.6))

In [52]:
metric_names = ['completed_trips',
 'requests_completed_rate',
#  'completed_to_session_ratio',
#  'requested_to_session_ratio',
 'gross_bookings_usd_finance',
 'gross_billings_usd_finance',
 'booking_to_hour_ratio',
 'variable_contribution_finance_48hours',
 'eyeball_eta',
 'avg_session_last_surge',
 'time_to_first_offer_per_trip',
 'offered_eta_per_offer',
 'network_eta_per_offer',
 'demand_jobs_request_to_pickup_time_per_trip',
 'demand_jobs_eta_per_assigned_job',
 'mean_demand_jobs_ata',
 'P90_demand_jobs_assigned_eta',
 'P90_demand_jobs_ata',
 'mean_atr',
 'driver_acceptance_rate',
 'driver_cancellation_rate',
 'unfulfilled_requests',
 'forward_dispatch_request_rate',
#  'demand_jobs_billable_distance',
 'demand_jobs_billable_duration',  
 'requesting_sessions_per_session',
#  'completed_trips_per_requesting_session',
# 'gross_unadjusted_billings_usd',
# 'rider_pre_dispatch_cancels_per_request',
'rider_post_dispatch_cancel_rate',
'rider_cancel_rate'
]

metric_names2 = ['completed_trips',
 'requests_completed_rate',
#  'completed_to_session_ratio',
#  'requested_to_session_ratio',
 'gross_bookings_usd_finance',
 'gross_billings_usd_finance',
#  'booking_to_hour_ratio',
#  'variable_contribution_finance_48hours',
#  'eyeball_eta',
#  'avg_session_last_surge',
#  'time_to_first_offer_per_trip',
 'offered_eta_per_offer',
 'network_eta_per_offer',
#  'demand_jobs_request_to_pickup_time_per_trip',
 'demand_jobs_eta_per_assigned_job',
#  'mean_demand_jobs_ata',
#  'P90_demand_jobs_assigned_eta',
#  'P90_demand_jobs_ata',
#  'mean_atr',
 'driver_acceptance_rate',
 'driver_cancellation_rate',
 'unfulfilled_requests',
#  'forward_dispatch_request_rate',
#  'demand_jobs_billable_distance',
#  'demand_jobs_billable_duration',  
#  'requesting_sessions_per_session',
#  'completed_trips_per_requesting_session',
# 'gross_unadjusted_billings_usd',
# 'rider_pre_dispatch_cancels_per_request',
'rider_post_dispatch_cancel_rate',
'rider_cancel_rate'
]

In [53]:
# xp_name = 'polysolve_partitioning_us'
xp_name = 'matching_metis_partitioning'
xp_info = MXBackendGateway.get_experiment_details_v2(xp_name)
print([city['id'] for city in xp_info['cities']])


[19, 1542, 140, 335, 41, 187, 136, 133, 13, 20, 21, 26, 1, 5, 198, 10]


In [54]:
x_and_marketplace_city_ids = [805, 148, 204, 781, 144, 146, 533, 1408, 1315, 1359, 1395, 1379, 1388, 1325, 1390, 1423, 1335, 1275]
x_and_marketplace_city_ids = [city['id'] for city in xp_info['cities']]

In [55]:
# subset_cities
# subset_cities = [805, 148, 204, 144, 533, 1408, 1315, 1359, 1395, 1379, 1388, 1325, 1390, 1423, 1335, 1275] # removed 2 cutues
# subset_cities = [148, 1408, 1379, 1388, 1390, 1423, 1335]
single_radar_cities = [133, 339, 228, 275, 341, 338, 317, 212, 19, 435, 390, 337, 284]

In [58]:
sb_overall_X = get_sb_for_segment_types(['product'], 
                                        {'product': ['UberX MASTER'], 
#                                          'city': subset_cities}, 
                                         'city': x_and_marketplace_city_ids}, 
#                                          'city': [204]}, 
                                        metric_names)
#                                         ['gross_bookings_usd_finance', 'avg_session_last_surge', 'requests_completed_rate'])
sb_overall_X.run_analysis()

06/22/2022 08:11:10 PM  You're using Queryrunner V2! 
06/22/2022 08:11:10 PM  [Polling] ef015d32-4e57-4e46-9fa2-8a0f03c76ebd 
06/22/2022 08:11:10 PM  [Status] created 
06/22/2022 08:11:12 PM  [Status] started validation 
06/22/2022 08:11:16 PM  [Status] started waiting to execute 
06/22/2022 08:11:20 PM  [Status] started execution 
06/22/2022 08:18:16 PM  [Status] completed success 
06/22/2022 08:18:16 PM  [Query Success] completed success 
06/22/2022 08:18:18 PM  You're using Queryrunner V2! 
06/22/2022 08:18:19 PM  [Polling] 67198c86-bb0c-4020-82ed-763bb43ca312 
06/22/2022 08:18:19 PM  [Status] created 
06/22/2022 08:18:20 PM  [Status] started validation 
06/22/2022 08:18:24 PM  [Status] started waiting to execute 
06/22/2022 08:18:28 PM  [Status] started execution 
06/22/2022 08:38:55 PM  [Status] completed success 
06/22/2022 08:38:55 PM  [Query Success] completed success 
06/22/2022 08:38:57 PM  You're using Queryrunner V2! 
06/22/2022 08:38:57 PM  [Polling] 5029ddc1-4e1b-4fdd-b28

2022-06-22 21:09:15,391 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[22]:C(city)[19], C(WOY)[24]:C(city)[198], C(WOY)[24]:C(city)[1], C(WOY)[24]:C(city)[136], C(WOY)[24]:C(city)[1542], C(WOY)[22]:C(city)[20], C(WOY)[24]:C(city)[133], C(WOY)[22]:C(city)[10], C(WOY)[22]:C(city)[335], C(WOY)[24]:C(city)[41], C(WOY)[22]:C(city)[140], C(WOY)[24]:C(city)[5], C(WOY)[24]:C(city)[13], C(WOY)[22]:C(city)[21], C(WOY)[22]:C(city)[187], C(WOY)[24]:C(city)[26]
2022-06-22 21:09:17,654 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[24.0]:C(city)[21], C(WOY)[24.0]:C(city)[140], C(WOY)[22.0]:C(city)[133], C(WOY)[24.0]:C(city)[20], C(WOY)[24.0]:C(city)[187], C(WOY)[24.0]:C(city)[10], C(WOY)[22.0]:C(city)[19], C(WOY)[24.0]:C(city)[41], C(WOY)[22.0]:C(city)[5], C(WOY)[24.0]:C(city)[1542], C(WOY)[24.0]:C(city)[13], C(WOY)[22.0]:C(city)[1], C(WOY)[24.0]:C(city)[198

2022-06-22 21:10:06,701 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[22]:C(city)[19], C(WOY)[24]:C(city)[198], C(WOY)[24]:C(city)[1], C(WOY)[24]:C(city)[136], C(WOY)[24]:C(city)[1542], C(WOY)[22]:C(city)[20], C(WOY)[24]:C(city)[133], C(WOY)[22]:C(city)[10], C(WOY)[22]:C(city)[335], C(WOY)[24]:C(city)[41], C(WOY)[22]:C(city)[140], C(WOY)[24]:C(city)[5], C(WOY)[24]:C(city)[13], C(WOY)[22]:C(city)[21], C(WOY)[22]:C(city)[187], C(WOY)[24]:C(city)[26]
2022-06-22 21:10:08,886 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[24.0]:C(city)[187], C(WOY)[22.0]:C(city)[198], C(WOY)[22.0]:C(city)[140], C(WOY)[24.0]:C(city)[10], C(WOY)[24.0]:C(city)[1], C(WOY)[24.0]:C(city)[21], C(WOY)[22.0]:C(city)[13], C(WOY)[24.0]:C(city)[335], C(WOY)[24.0]:C(city)[41], C(WOY)[22.0]:C(city)[19], C(WOY)[22.0]:C(city)[5], C(WOY)[24.0]:C(city)[136], C(WOY)[22.0]:C(city)[133

2022-06-22 21:10:57,228 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[22]:C(city)[19], C(WOY)[24]:C(city)[198], C(WOY)[24]:C(city)[1], C(WOY)[24]:C(city)[136], C(WOY)[24]:C(city)[1542], C(WOY)[22]:C(city)[20], C(WOY)[24]:C(city)[133], C(WOY)[22]:C(city)[10], C(WOY)[22]:C(city)[335], C(WOY)[24]:C(city)[41], C(WOY)[22]:C(city)[140], C(WOY)[24]:C(city)[5], C(WOY)[24]:C(city)[13], C(WOY)[22]:C(city)[21], C(WOY)[22]:C(city)[187], C(WOY)[24]:C(city)[26]
2022-06-22 21:10:59,804 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[24.0]:C(city)[21], C(WOY)[24.0]:C(city)[10], C(WOY)[24.0]:C(city)[26], C(WOY)[24.0]:C(city)[13], C(WOY)[24.0]:C(city)[140], C(WOY)[24.0]:C(city)[1], C(WOY)[24.0]:C(city)[20], C(WOY)[24.0]:C(city)[198], C(WOY)[24.0]:C(city)[136], C(WOY)[24.0]:C(city)[5], C(WOY)[24.0]:C(city)[335], C(WOY)[24.0]:C(city)[187], C(WOY)[24.0]:C(city)[133

LinAlgError: Singular matrix

In [60]:
sb_overall_X.data_df['city'].value_counts()

335     840
1542    840
198     840
133     840
21      840
13      840
5       840
140     840
20      840
187     840
19      840
26      840
10      840
41      840
1       840
136     840
Name: city, dtype: int64

In [57]:
sb_overall_X.results_table()

,Metric,City,Product,Num. Obs.,Predicted Control Value,Predicted Treatment Value,Rel. Effect,Rel. Eff. Std. Err.,95% Rel. Eff. CI LB,95% Rel. Eff. CI UB,Level Effect,Level Std. Err.,95% Level Eff. CI LB,95% Level Eff. CI UB,Significant?,p
0,Gross Bookings (USD) per Hour,"[1, 5, 1542, 198, 136, 41, 10, 133, 140, 13, 3...",[UberX],6630,68937.79,69158.81,0.32%,0.10%,0.12%,0.52%,221.01,69.91,83.96,358.06,True,0.00
1,Average Last Surge Per Session,"[1, 5, 1542, 198, 136, 41, 10, 133, 140, 13, 3...",[UberX],6686,1.16,1.16,-0.02%,0.16%,-0.33%,0.28%,-0.00,0.00,-0.00,0.00,False,0.88
2,Completed Trips Per Request,"[1, 5, 1542, 198, 136, 41, 10, 133, 140, 13, 3...",[UberX],6686,0.89,0.89,0.03%,0.04%,-0.05%,0.11%,0.00,0.00,-0.00,0.00,False,0.49


In [66]:
sb_overall_X.results_table(decimals=4).to_csv('polysolve_natural_partitioning_us.csv')

In [24]:
sb_overall_X.results_table()

,Metric,City,Product,Num. Obs.,Predicted Control Value,Predicted Treatment Value,Rel. Effect,Rel. Eff. Std. Err.,95% Rel. Eff. CI LB,95% Rel. Eff. CI UB,Level Effect,Level Std. Err.,95% Level Eff. CI LB,95% Level Eff. CI UB,Significant?,p
0,Completed Trips per Hour,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2921,979.02,978.46,-0.06%,0.10%,-0.26%,0.14%,-0.57,1.00,-2.53,1.40,False,0.57
1,Completed Trips Per Request,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2913,0.73,0.73,0.02%,0.12%,-0.22%,0.26%,0.00,0.00,-0.00,0.00,False,0.86
2,Completed Trips Per Session,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2921,0.54,0.54,0.01%,0.13%,-0.25%,0.26%,0.00,0.00,-0.00,0.00,False,0.95
3,Requests Per Session,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2901,0.74,0.74,0.03%,0.15%,-0.27%,0.33%,0.00,0.00,-0.00,0.00,False,0.86
4,Gross Bookings (USD) per Hour,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2921,4234.49,4249.15,0.35%,0.19%,-0.02%,0.71%,14.66,7.84,-0.72,30.03,False,0.06
5,Gross Billings (USD) per Hour,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2921,4196.76,4213.96,0.41%,0.19%,0.03%,0.79%,17.20,8.06,1.39,33.01,True,0.03
6,Gross Bookings Per Supply Hour (USD),"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2907,6.53,6.54,0.20%,0.35%,-0.49%,0.89%,0.01,0.02,-0.03,0.06,False,0.57
7,Variable Contribution 48 Hour (USD) per Hour,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2919,1216.54,1221.57,0.41%,0.19%,0.04%,0.78%,5.03,2.29,0.54,9.52,True,0.03
8,Mean Eyeball ETA,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2913,319.75,319.01,-0.23%,0.14%,-0.51%,0.05%,-0.74,0.45,-1.63,0.15,False,0.10
9,Average Last Surge Per Session,"[1408, 1379, 1388, 1390, 1423, 148, 1335]",[UberX],2907,1.11,1.11,-0.07%,0.14%,-0.35%,0.20%,-0.00,0.00,-0.00,0.00,False,0.61


In [ ]:
sb_city_X = get_sb_for_segment_types(['product', 'city'], 
                                     {'product': ['UberX MASTER'], 
                                      'city': x_and_marketplace_city_ids}, 
                                     metric_names)
sb_city_X.run_analysis()

2022-05-10 01:16:22,384 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16]:C(city)[284], C(WOY)[16]:C(city)[212], C(WOY)[16]:C(city)[339], C(WOY)[18]:C(city)[19], C(WOY)[18]:C(city)[133], C(WOY)[16]:C(city)[5], C(WOY)[16]:C(city)[435], C(WOY)[16]:C(city)[198], C(WOY)[18]:C(city)[20], C(WOY)[18]:C(city)[315], C(WOY)[16]:C(city)[188], C(WOY)[16]:C(city)[40], C(WOY)[16]:C(city)[228], C(WOY)[16]:C(city)[192], C(WOY)[18]:C(city)[41], C(WOY)[18]:C(city)[338], C(WOY)[18]:C(city)[390], C(WOY)[18]:C(city)[341], C(WOY)[18]:C(city)[88], C(WOY)[18]:C(city)[337], C(WOY)[16]:C(city)[317], C(WOY)[18]:C(city)[275]
2022-05-10 01:16:28,758 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[88], C(WOY)[18.0]:C(city)[228], C(WOY)[18.0]:C(city)[284], C(WOY)[16.0]:C(city)[188], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[19], C(WOY)[18.0]:C(city)[192],

2022-05-10 01:16:51,832 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[228], C(WOY)[18.0]:C(city)[275], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[212], C(WOY)[18.0]:C(city)[19], C(WOY)[18.0]:C(city)[5], C(WOY)[18.0]:C(city)[133], C(WOY)[18.0]:C(city)[341], C(WOY)[16.0]:C(city)[40], C(WOY)[16.0]:C(city)[338], C(WOY)[18.0]:C(city)[188], C(WOY)[18.0]:C(city)[435], C(WOY)[16.0]:C(city)[337], C(WOY)[18.0]:C(city)[198], C(WOY)[18.0]:C(city)[390], C(WOY)[16.0]:C(city)[317], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[192], C(WOY)[16.0]:C(city)[315], C(WOY)[18.0]:C(city)[88], C(WOY)[16.0]:C(city)[284], C(WOY)[18.0]:C(city)[20]
2022-05-10 01:16:53,131 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:16:53,264 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearit

2022-05-10 01:17:15,290 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:17:15,424 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:17:15,707 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:17:15,850 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:17:16,111 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:17:16,244 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:17:16,521 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of c

2022-05-10 01:17:37,368 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:17:37,632 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:17:37,762 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:17:38,001 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:17:38,128 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:17:38,373 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:17:38,503 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because 

2022-05-10 01:18:00,652 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:18:00,789 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:18:01,050 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:18:01,255 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18.0]
2022-05-10 01:18:01,526 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:18:01,664 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:18:01,939 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of c

...

2022-05-10 01:20:25,413 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:20:25,546 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:20:25,810 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:20:25,941 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:20:26,194 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:26,325 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:20:26,592 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because 

2022-05-10 01:20:49,493 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:49,757 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:49,890 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:50,151 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:50,287 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:50,547 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:20:50,678 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of col

2022-05-10 01:21:13,336 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:13,472 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:13,770 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:13,901 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18.0]
2022-05-10 01:21:14,169 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:14,302 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:21:14,561 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of c

2022-05-10 01:21:37,036 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:37,296 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:37,427 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:21:37,689 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:21:37,820 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16.0]
2022-05-10 01:21:38,087 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:21:38,217 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of

2022-05-10 01:22:02,125 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[16]
2022-05-10 01:22:02,409 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18]
2022-05-10 01:22:02,541 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[18.0]


In [71]:
sb_city_X.results_table()

,Metric,City,Product,Num. Obs.,Predicted Control Value,Predicted Treatment Value,Rel. Effect,Rel. Eff. Std. Err.,95% Rel. Eff. CI LB,95% Rel. Eff. CI UB,Level Effect,Level Std. Err.,95% Level Eff. CI LB,95% Level Eff. CI UB,Significant?,p
0,Completed Trips per Hour,"[5, 19, 20, 40, 41, 88, 133, 188, 192, 198, 21...",[UberX],9228,1502.89,1501.83,-0.07%,0.13%,-0.32%,0.18%,-1.06,1.88,-4.75,2.63,False,0.57
1,Completed Trips per Hour,[5],[UberX],420,15611.89,15588.76,-0.15%,0.22%,-0.57%,0.28%,-23.13,33.74,-89.45,43.20,False,0.49
2,Completed Trips per Hour,[19],[UberX],420,1653.76,1652.82,-0.06%,0.26%,-0.57%,0.46%,-0.94,4.32,-9.43,7.55,False,0.83
3,Completed Trips per Hour,[20],[UberX],420,3211.23,3218.37,0.22%,0.19%,-0.16%,0.60%,7.14,6.17,-4.98,19.27,False,0.25
4,Completed Trips per Hour,[40],[UberX],420,479.94,477.49,-0.51%,0.60%,-1.69%,0.67%,-2.45,2.87,-8.09,3.20,False,0.39
5,Completed Trips per Hour,[41],[UberX],420,542.71,541.52,-0.22%,0.85%,-1.89%,1.45%,-1.19,4.61,-10.26,7.88,False,0.80
6,Completed Trips per Hour,[88],[UberX],418,521.80,527.92,1.17%,0.45%,0.29%,2.06%,6.12,2.35,1.49,10.75,True,0.01
7,Completed Trips per Hour,[133],[UberX],420,558.05,558.28,0.04%,0.50%,-0.94%,1.03%,0.23,2.79,-5.27,5.72,False,0.94
8,Completed Trips per Hour,[188],[UberX],420,1022.28,1019.01,-0.32%,0.44%,-1.19%,0.55%,-3.27,4.52,-12.15,5.61,False,0.47
9,Completed Trips per Hour,[192],[UberX],418,763.56,763.94,0.05%,0.34%,-0.62%,0.72%,0.38,2.61,-4.75,5.52,False,0.88


In [69]:
sb_peak_X = get_sb_for_segment_types(['product', 'peak'], 
                                     {'product': ['UberX MASTER'], 
                                      'city': x_and_marketplace_city_ids}, 
                                     metric_names)
sb_peak_X.run_analysis()

2022-05-10 01:32:18,521 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[228], C(WOY)[18.0]:C(city)[275], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[212], C(WOY)[18.0]:C(city)[19], C(WOY)[18.0]:C(city)[5], C(WOY)[18.0]:C(city)[133], C(WOY)[18.0]:C(city)[341], C(WOY)[16.0]:C(city)[40], C(WOY)[16.0]:C(city)[338], C(WOY)[18.0]:C(city)[188], C(WOY)[18.0]:C(city)[435], C(WOY)[16.0]:C(city)[337], C(WOY)[18.0]:C(city)[198], C(WOY)[18.0]:C(city)[390], C(WOY)[16.0]:C(city)[317], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[192], C(WOY)[16.0]:C(city)[315], C(WOY)[18.0]:C(city)[88], C(WOY)[16.0]:C(city)[284], C(WOY)[18.0]:C(city)[20]
2022-05-10 01:32:19,037 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[17.0]:C(city)[188]
2022-05-10 01:32:25,918 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix becaus

2022-05-10 01:33:34,833 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[212], C(WOY)[16.0]:C(city)[88], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[5], C(WOY)[16.0]:C(city)[275], C(WOY)[16.0]:C(city)[188], C(WOY)[18.0]:C(city)[317], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[284], C(WOY)[18.0]:C(city)[20], C(WOY)[16.0]:C(city)[192], C(WOY)[18.0]:C(city)[390], C(WOY)[18.0]:C(city)[19], C(WOY)[16.0]:C(city)[133], C(WOY)[18.0]:C(city)[228], C(WOY)[18.0]:C(city)[198], C(WOY)[18.0]:C(city)[338], C(WOY)[16.0]:C(city)[435], C(WOY)[18.0]:C(city)[341], C(WOY)[16.0]:C(city)[315], C(WOY)[18.0]:C(city)[40], C(WOY)[16.0]:C(city)[41]
2022-05-10 01:33:40,509 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[212], C(WOY)[16.0]:C(city)[88], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[5], C(WOY)[16.0]:C(city)[275], C(WOY)[16.

2022-05-10 01:34:54,825 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[317], C(WOY)[16.0]:C(city)[337], C(WOY)[16.0]:C(city)[284], C(WOY)[18.0]:C(city)[40], C(WOY)[18.0]:C(city)[19], C(WOY)[16.0]:C(city)[339], C(WOY)[16.0]:C(city)[212], C(WOY)[16.0]:C(city)[188], C(WOY)[18.0]:C(city)[88], C(WOY)[16.0]:C(city)[192], C(WOY)[18.0]:C(city)[133], C(WOY)[18.0]:C(city)[228], C(WOY)[16.0]:C(city)[390], C(WOY)[18.0]:C(city)[338], C(WOY)[18.0]:C(city)[341], C(WOY)[16.0]:C(city)[5], C(WOY)[16.0]:C(city)[315], C(WOY)[18.0]:C(city)[435], C(WOY)[18.0]:C(city)[20], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[198], C(WOY)[16.0]:C(city)[275]
2022-05-10 01:35:02,775 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16]:C(city)[284], C(WOY)[16]:C(city)[212], C(WOY)[16]:C(city)[339], C(WOY)[18]:C(city)[19], C(WOY)[18]:C(city)[133], C(WOY)[16]:C(city)

2022-05-10 01:36:16,084 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[228], C(WOY)[18.0]:C(city)[317], C(WOY)[18.0]:C(city)[5], C(WOY)[18.0]:C(city)[20], C(WOY)[18.0]:C(city)[212], C(WOY)[16.0]:C(city)[88], C(WOY)[18.0]:C(city)[338], C(WOY)[16.0]:C(city)[40], C(WOY)[16.0]:C(city)[275], C(WOY)[18.0]:C(city)[284], C(WOY)[18.0]:C(city)[133], C(WOY)[16.0]:C(city)[19], C(WOY)[16.0]:C(city)[341], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[390], C(WOY)[18.0]:C(city)[198], C(WOY)[16.0]:C(city)[315], C(WOY)[16.0]:C(city)[192], C(WOY)[18.0]:C(city)[435], C(WOY)[18.0]:C(city)[41], C(WOY)[16.0]:C(city)[188], C(WOY)[16.0]:C(city)[337]
2022-05-10 01:36:22,944 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[228], C(WOY)[18.0]:C(city)[317], C(WOY)[18.0]:C(city)[5], C(WOY)[18.0]:C(city)[20], C(WOY)[18.0]:C(city)[212], C(WOY)[16.

2022-05-10 01:37:38,584 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[212], C(WOY)[16.0]:C(city)[228], C(WOY)[18.0]:C(city)[317], C(WOY)[18.0]:C(city)[284], C(WOY)[18.0]:C(city)[19], C(WOY)[16.0]:C(city)[275], C(WOY)[18.0]:C(city)[133], C(WOY)[18.0]:C(city)[435], C(WOY)[16.0]:C(city)[390], C(WOY)[16.0]:C(city)[5], C(WOY)[18.0]:C(city)[188], C(WOY)[16.0]:C(city)[338], C(WOY)[16.0]:C(city)[341], C(WOY)[16.0]:C(city)[192], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[198], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[88], C(WOY)[18.0]:C(city)[315], C(WOY)[16.0]:C(city)[41], C(WOY)[16.0]:C(city)[40], C(WOY)[16.0]:C(city)[20]
2022-05-10 01:37:38,798 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[17.0]:C(city)[188]
2022-05-10 01:37:46,739 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix becaus

...

2022-05-10 01:49:49,176 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[228], C(WOY)[18.0]:C(city)[40], C(WOY)[18.0]:C(city)[188], C(WOY)[18.0]:C(city)[88], C(WOY)[16.0]:C(city)[192], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[133], C(WOY)[16.0]:C(city)[338], C(WOY)[18.0]:C(city)[339], C(WOY)[18.0]:C(city)[5], C(WOY)[16.0]:C(city)[275], C(WOY)[18.0]:C(city)[315], C(WOY)[18.0]:C(city)[284], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[19], C(WOY)[18.0]:C(city)[435], C(WOY)[16.0]:C(city)[198], C(WOY)[16.0]:C(city)[341], C(WOY)[16.0]:C(city)[390], C(WOY)[18.0]:C(city)[212], C(WOY)[18.0]:C(city)[317], C(WOY)[16.0]:C(city)[20]
2022-05-10 01:49:54,265 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[228], C(WOY)[18.0]:C(city)[40], C(WOY)[18.0]:C(city)[188], C(WOY)[18.0]:C(city)[88], C(WOY)[16.0]:C(city)[192], C(WOY)[18

2022-05-10 01:51:10,799 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[88], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[133], C(WOY)[18.0]:C(city)[390], C(WOY)[16.0]:C(city)[338], C(WOY)[18.0]:C(city)[20], C(WOY)[16.0]:C(city)[284], C(WOY)[18.0]:C(city)[40], C(WOY)[16.0]:C(city)[315], C(WOY)[16.0]:C(city)[5], C(WOY)[16.0]:C(city)[212], C(WOY)[16.0]:C(city)[228], C(WOY)[16.0]:C(city)[275], C(WOY)[16.0]:C(city)[341], C(WOY)[16.0]:C(city)[19], C(WOY)[18.0]:C(city)[192], C(WOY)[18.0]:C(city)[435], C(WOY)[18.0]:C(city)[41], C(WOY)[16.0]:C(city)[339], C(WOY)[18.0]:C(city)[198], C(WOY)[16.0]:C(city)[317], C(WOY)[16.0]:C(city)[188]
2022-05-10 01:51:18,200 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16]:C(city)[284], C(WOY)[16]:C(city)[212], C(WOY)[16]:C(city)[339], C(WOY)[18]:C(city)[19], C(WOY)[18]:C(city)[133], C(WOY)[16]:C(city)

2022-05-10 01:52:30,169 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[19], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[338], C(WOY)[16.0]:C(city)[88], C(WOY)[16.0]:C(city)[212], C(WOY)[16.0]:C(city)[20], C(WOY)[18.0]:C(city)[317], C(WOY)[16.0]:C(city)[188], C(WOY)[16.0]:C(city)[5], C(WOY)[16.0]:C(city)[390], C(WOY)[18.0]:C(city)[339], C(WOY)[18.0]:C(city)[228], C(WOY)[18.0]:C(city)[133], C(WOY)[16.0]:C(city)[40], C(WOY)[18.0]:C(city)[275], C(WOY)[18.0]:C(city)[192], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[315], C(WOY)[16.0]:C(city)[284], C(WOY)[18.0]:C(city)[198], C(WOY)[16.0]:C(city)[341], C(WOY)[16.0]:C(city)[435]
2022-05-10 01:52:36,158 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[16.0]:C(city)[19], C(WOY)[18.0]:C(city)[337], C(WOY)[18.0]:C(city)[338], C(WOY)[16.0]:C(city)[88], C(WOY)[16.0]:C(city)[212], C(WOY)[16

2022-05-10 01:53:50,283 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[317], C(WOY)[16.0]:C(city)[88], C(WOY)[16.0]:C(city)[212], C(WOY)[16.0]:C(city)[338], C(WOY)[16.0]:C(city)[133], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[390], C(WOY)[16.0]:C(city)[284], C(WOY)[18.0]:C(city)[339], C(WOY)[18.0]:C(city)[192], C(WOY)[18.0]:C(city)[275], C(WOY)[16.0]:C(city)[5], C(WOY)[16.0]:C(city)[337], C(WOY)[16.0]:C(city)[315], C(WOY)[18.0]:C(city)[341], C(WOY)[18.0]:C(city)[19], C(WOY)[16.0]:C(city)[40], C(WOY)[16.0]:C(city)[228], C(WOY)[18.0]:C(city)[198], C(WOY)[18.0]:C(city)[188], C(WOY)[16.0]:C(city)[435], C(WOY)[16.0]:C(city)[20]
2022-05-10 01:53:50,542 mxpkg.stats.switchback_utils WARNING Dropping the following column from design matrix because of collinearity: C(WOY)[17.0]:C(city)[188]
2022-05-10 01:53:57,954 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix becaus

2022-05-10 01:55:14,896 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[337], C(WOY)[16.0]:C(city)[275], C(WOY)[18.0]:C(city)[228], C(WOY)[16.0]:C(city)[20], C(WOY)[18.0]:C(city)[5], C(WOY)[16.0]:C(city)[317], C(WOY)[18.0]:C(city)[133], C(WOY)[18.0]:C(city)[88], C(WOY)[18.0]:C(city)[212], C(WOY)[18.0]:C(city)[40], C(WOY)[16.0]:C(city)[390], C(WOY)[18.0]:C(city)[284], C(WOY)[18.0]:C(city)[19], C(WOY)[18.0]:C(city)[41], C(WOY)[18.0]:C(city)[315], C(WOY)[18.0]:C(city)[198], C(WOY)[16.0]:C(city)[341], C(WOY)[18.0]:C(city)[338], C(WOY)[18.0]:C(city)[339], C(WOY)[16.0]:C(city)[188], C(WOY)[16.0]:C(city)[192], C(WOY)[16.0]:C(city)[435]
2022-05-10 01:55:22,048 mxpkg.stats.switchback_utils WARNING Dropping the following columns from design matrix because of collinearity: C(WOY)[18.0]:C(city)[337], C(WOY)[16.0]:C(city)[275], C(WOY)[18.0]:C(city)[228], C(WOY)[16.0]:C(city)[20], C(WOY)[18.0]:C(city)[5], C(WOY)[16.

In [70]:
sb_peak_X.results_table()


,Metric,City,Product,Peak,Num. Obs.,Predicted Control Value,Predicted Treatment Value,Rel. Effect,Rel. Eff. Std. Err.,95% Rel. Eff. CI LB,95% Rel. Eff. CI UB,Level Effect,Level Std. Err.,95% Level Eff. CI LB,95% Level Eff. CI UB,Significant?,p
0,Completed Trips per Hour,"[5, 19, 20, 40, 41, 88, 133, 188, 192, 198, 21...",[UberX],"[off\_peak, peak]",9228,1502.89,1501.83,-0.07%,0.13%,-0.32%,0.18%,-1.06,1.88,-4.75,2.63,False,0.57
1,Completed Trips per Hour,"[5, 390, 133, 19, 20, 275, 284, 40, 41, 435, 3...",[UberX],[off\_peak],6416,1068.30,1068.23,-0.01%,0.18%,-0.36%,0.35%,-0.06,1.92,-3.83,3.70,False,0.97
2,Completed Trips per Hour,"[5, 390, 133, 19, 20, 275, 284, 40, 41, 435, 3...",[UberX],[peak],2812,2523.25,2519.91,-0.13%,0.17%,-0.47%,0.21%,-3.34,4.39,-11.95,5.26,False,0.45
3,Completed Trips Per Request,"[5, 19, 20, 40, 41, 88, 133, 188, 192, 198, 21...",[UberX],"[off\_peak, peak]",9172,0.89,0.89,-0.00%,0.06%,-0.11%,0.10%,-0.00,0.00,-0.00,0.00,False,0.95
4,Completed Trips Per Request,"[5, 390, 133, 19, 20, 275, 284, 40, 41, 435, 3...",[UberX],[off\_peak],6350,0.89,0.89,-0.01%,0.07%,-0.14%,0.13%,-0.00,0.00,-0.00,0.00,False,0.93
5,Completed Trips Per Request,"[5, 390, 133, 19, 20, 275, 284, 40, 41, 435, 3...",[UberX],[peak],2822,0.89,0.89,-0.02%,0.09%,-0.19%,0.15%,-0.00,0.00,-0.00,0.00,False,0.82
6,Completed Trips Per Session,"[5, 19, 20, 40, 41, 88, 133, 188, 192, 198, 21...",[UberX],"[off\_peak, peak]",9214,0.55,0.55,-0.04%,0.14%,-0.31%,0.23%,-0.00,0.00,-0.00,0.00,False,0.77
7,Completed Trips Per Session,"[5, 390, 133, 19, 20, 275, 284, 40, 41, 435, 3...",[UberX],[off\_peak],6394,0.55,0.55,0.07%,0.19%,-0.29%,0.44%,0.00,0.00,-0.00,0.00,False,0.69
8,Completed Trips Per Session,"[5, 390, 133, 19, 20, 275, 284, 40, 41, 435, 3...",[UberX],[peak],2820,0.55,0.55,-0.13%,0.18%,-0.49%,0.23%,-0.00,0.00,-0.00,0.00,False,0.47
9,Requests Per Session,"[5, 19, 20, 40, 41, 88, 133, 188, 192, 198, 21...",[UberX],"[off\_peak, peak]",9200,0.62,0.62,-0.01%,0.13%,-0.26%,0.24%,-0.00,0.00,-0.00,0.00,False,0.96


In [73]:
sb_peak_X.results_table(decimals=4).to_csv('polysolve_natural_partitioning_us_peak.csv')
sb_city_X.results_table(decimals=4).to_csv('polysolve_natural_partitioning_us_city.csv')

In [67]:
# sb_peak_X.results_table(decimals=4).to_csv('polysolve_latam_peak_Jan22.csv')

In [68]:
# sb_city_X.results_table(decimals=4).to_csv('polysolve_latam_cities_Jan22.csv')

In [29]:
import pandas as pd
sb_city_X = pd.read_csv('polysolve_latam_cities_Jan22.csv')
sb_city_X['Metric'][:20]
# sb_city_X.query("CAST(City AS int) == 146")
options = ["['1388']"]
metric_options = ['Completed Trips per Hour', 'Completed Trips Per Request', 'Gross Billings (USD) per Hour', 'Variable Contribution 48 Hour (USD) per Hour', 'Gross Unadjusted Billings (USD) per Hour']
sb_city_X.loc[sb_city_X['City'].isin(options)]#.loc[sb_city_X['City'].isin(metric_options)]
# sb_city_X.loc[sb_city_X['City'].isin(metric_options)]

,Unnamed: 0,Metric,City,Product,Num. Obs.,Predicted Control Value,Predicted Treatment Value,Rel. Effect,Rel. Eff. Std. Err.,95% Rel. Eff. CI LB,95% Rel. Eff. CI UB,Level Effect,Level Std. Err.,95% Level Eff. CI LB,95% Level Eff. CI UB,Significant?,p
14,14,Completed Trips per Hour,['1388'],['UberX'],420,580.0072,578.5361,-0.2536%,0.2950%,-0.8335%,0.3262%,-1.4711,1.7109,-4.8342,1.8920,False,0.3904
33,33,Completed Trips Per Request,['1388'],['UberX'],420,0.6831,0.6850,0.2883%,0.5663%,-0.8249%,1.4014%,0.0020,0.0039,-0.0056,0.0096,False,0.6110
52,52,Completed Trips Per Session,['1388'],['UberX'],418,0.6477,0.6486,0.1417%,0.4323%,-0.7081%,0.9915%,0.0009,0.0028,-0.0046,0.0064,False,0.7432
71,71,Requests Per Session,['1388'],['UberX'],416,0.9485,0.9459,-0.2666%,0.5965%,-1.4393%,0.9061%,-0.0025,0.0057,-0.0137,0.0086,False,0.6551
90,90,Gross Bookings (USD) per Hour,['1388'],['UberX'],420,1166.9631,1170.2560,0.2822%,0.6182%,-0.9331%,1.4974%,3.2929,7.2144,-10.8886,17.4745,False,0.6483
109,109,Gross Billings (USD) per Hour,['1388'],['UberX'],420,1096.4604,1099.9476,0.3180%,0.5808%,-0.8237%,1.4597%,3.4872,6.3683,-9.0312,16.0056,False,0.5843
128,128,Gross Bookings Per Supply Hour (USD),['1388'],['UberX'],418,5.8000,5.8154,0.2657%,1.0084%,-1.7165%,2.2480%,0.0154,0.0585,-0.0996,0.1304,False,0.7923
147,147,Variable Contribution 48 Hour (USD) per Hour,['1388'],['UberX'],418,244.5162,245.4915,0.3989%,0.4478%,-0.4814%,1.2791%,0.9753,1.0949,-1.1771,3.1276,False,0.3736
166,166,Mean Eyeball ETA,['1388'],['UberX'],416,283.3001,283.1075,-0.0680%,0.3770%,-0.8090%,0.6731%,-0.1926,1.0679,-2.2920,1.9069,False,0.8570
185,185,Average Last Surge Per Session,['1388'],['UberX'],420,1.1597,1.1623,0.2221%,0.3875%,-0.5396%,0.9838%,0.0026,0.0045,-0.0063,0.0114,False,0.5668
